### Time Series Workshop 
# 3. Air Pollutants &#x1F525;: Forecasting

In this notebook, we will use the preprocessed data from the feature engineering notebook and, finally, perform some forecasting!

In [2]:
%config InlineBackend.figure_format='retina'
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

DATA_DIR = Path("..") / Path("data")

## Load data

In [7]:
FILE_PATH = DATA_DIR / "air_quality_processed.csv"

df_in = pd.read_csv(
    FILE_PATH, 
    parse_dates=["date_time"],
    index_col=["date_time"],
)
df_in.head()

,co_sensor,month,week,day,day_of_week,hour,is_weekend,co_sensor_lag_1,co_sensor_lag_2,co_sensor_lag_3,...,co_sensor_win_max,co_sensor_win_std,humidity_win_mean,humidity_win_min,humidity_win_max,humidity_win_std,month_sin,month_cos,hour_sin,hour_cos
date_time,,,,,,,,,,,,,,,,,,,,,
2004-04-05 00:00:00,1065.0,4,15,5,0,0,0,1188.0,1196.0,1113.0,...,1420.0,130.064599,53.84,44.6,60.8,6.854415,0.866025,-0.5,0.000000,1.000000
2004-04-05 01:00:00,999.0,4,15,5,0,1,0,1065.0,1188.0,1196.0,...,1366.0,114.535147,58.08,48.9,65.8,6.239151,0.866025,-0.5,0.269797,0.962917
2004-04-05 02:00:00,911.0,4,15,5,0,2,0,999.0,1065.0,1188.0,...,1196.0,83.382852,64.14,56.1,79.2,9.136082,0.866025,-0.5,0.519584,0.854419
2004-04-05 03:00:00,873.0,4,15,5,0,3,0,911.0,999.0,1065.0,...,1196.0,122.607912,68.92,58.8,80.0,10.081270,0.866025,-0.5,0.730836,0.682553
2004-04-05 04:00:00,881.0,4,15,5,0,4,0,873.0,911.0,999.0,...,1188.0,125.857856,73.36,60.8,81.0,9.373793,0.866025,-0.5,0.887885,0.460065
